In [ ]:
import pandas as pd
import html

# Show all columns without wrapping
pd.set_option('display.max_columns', None)

In [ ]:
import netifaces

gateways = netifaces.gateways()
default_gateway = gateways.get('default', {}).get(netifaces.AF_INET)
print(gateways)
if default_gateway:
    router_ip = default_gateway[0]
    print(f"The router IP is likely: {router_ip}")
else:
    print("Could not find the default gateway address.")


In [ ]:
import subprocess

# Suppose your router IP is 192.168.1.1
router_ip = "192.168.100.1"

result = subprocess.run(["ping", "-c", "1", router_ip], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if result.returncode == 0:
    print(f"Successfully reached {router_ip}")
else:
    print(f"Could not reach {router_ip}")


In [ ]:
import socket

router_ip = "192.168.100.1"
port = 80  # Common for router web admin interface, though some may use 443 or another port

try:
    s = socket.create_connection((router_ip, port), timeout=2)
    print(f"Connected to {router_ip}:{port}")
    s.close()
except (socket.timeout, socket.error) as e:
    print(f"Could not connect to {router_ip}:{port} - {e}")


## Login to the router web interface

In [ ]:
import requests
import random
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_v1_5
from base64 import b64encode
import os
import json

# Load secrets
secrets_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "secrets", "secrets.json")

try:
    with open(secrets_path, "r") as secrets_file:
        secrets = json.load(secrets_file)
except FileNotFoundError:
    raise FileNotFoundError(f"Secrets file not found at {secrets_path}")

router_ip = secrets["router_ip"]
username = secrets["username"]
plaintext_password = secrets["password"]
modulus_hex = secrets["modulus_hex"]

router_ip = "192.168.100.1"
username = "admin"
plaintext_password = "v2jm5tr6"  # Replace with your actual plaintext password

session = requests.Session()

# GET to establish cookies
resp = session.get(f"http://{router_ip}/logout.cgi", verify=False)

# Generate a sessionId
session_id = random.randint(1, 799999999)

# RSA Encryption of password
modulus_int = int(modulus_hex, 16)
exponent_int = int("10001", 16)  # 65537 in decimal

rsa_key = RSA.construct((modulus_int, exponent_int))
cipher = PKCS1_v1_5.new(rsa_key)
encrypted_pass = cipher.encrypt(plaintext_password.encode('utf-8'))
submitext = b64encode(encrypted_pass).decode('utf-8')

payload = {
    "sessionKey": "0",
    "sessionId": str(session_id),
    "inputUserName": username,
    "inputPassword": submitext,
    "nothankyou": "1"
}

headers = {
    "Host": router_ip,
    "Origin": f"http://{router_ip}",
    "Referer": f"http://{router_ip}/logout.cgi",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,"
              "image/avif,image/webp,image/apng,*/*;q=0.8,"
              "application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Content-Type": "application/x-www-form-urlencoded",
    "Upgrade-Insecure-Requests": "1",
    "Cache-Control": "max-age=0"
}

login_resp = session.post(f"http://{router_ip}/login.cgi", data=payload, headers=headers, verify=False)

if "Logout" in login_resp.text or login_resp.status_code == 200:
    print("Login successful!")
    print(login_resp.status_code)
    print("Login response text: \n",login_resp.text)
else:
    print("Login might have failed. Check the response:")
    print(login_resp.text)


## Understanding the ARP Table

An **ARP (Address Resolution Protocol) table** is a data structure used by devices on a local network to map **IP addresses** to **MAC (Media Access Control) addresses**.

### How It Works in Practice

1. **Scenario**: Your computer wants to send data to `192.168.1.50`.

2. **Check ARP Table**:
   - Before sending the packet at the Ethernet layer, the computer checks its ARP table to see if it already knows the MAC address for that IP.
   - If the entry is found, the computer sends the Ethernet frame directly to the corresponding MAC address.

3. **No Entry Found**:
   - If the ARP table doesn’t contain the MAC address for the IP, the computer broadcasts an **ARP request** packet on the local network:  
     *“Who has 192.168.1.50?”*

4. **Response**:
   - The device with the IP address `192.168.1.50` responds with its MAC address.

5. **Update ARP Table**:
   - The computer updates its ARP table with the newly learned IP-to-MAC mapping.

6. **Future Communication**:
   - For subsequent communications with `192.168.1.50`, the computer can now send packets directly to the known MAC address without repeating the ARP query.

This process ensures efficient communication within a local network by caching IP-to-MAC mappings.


In [ ]:
import pandas as pd
import requests

def extract_table_data(session, table_url, marker_start, marker_end, parse_logic):
    """
    Extracts table data from a specified URL and parses it into a DataFrame.

    Args:
        session (requests.Session): The logged-in session object.
        table_url (str): The URL of the table to extract.
        parse_logic (function): A function that parses individual entries.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted table data.
    """
    response = session.get(table_url, verify=False)
    if response.status_code == 200:
        # Extract the HTML content
        page_content = response.text

        # Locate the JavaScript variable 'init_value'
        start_idx = page_content.find(marker_start) + len(marker_start)
        end_idx = page_content.find(marker_end, start_idx)

        if start_idx > len(marker_start) and end_idx > start_idx:
            init_value = page_content[start_idx:end_idx]
            
            # Parse the table entries using the provided parsing logic
            table_entries = []
            for entry in init_value.split("|"):
                if entry.strip():  # Ignore empty entries
                    parsed_entry = parse_logic(entry)
                    if parsed_entry:  # Only add valid entries
                        table_entries.append(parsed_entry)
            
            # Return as a DataFrame
            return pd.DataFrame(table_entries)
        else:
            print("Failed to extract table data from the page.")
    else:
        print(f"Failed to fetch the table page. Status code: {response.status_code}")
    return pd.DataFrame()  # Return an empty DataFrame if parsing fails

find mac vendor based on downloaded mac vendors csv file

Parse Logic for all tables

In [ ]:
# Define parse logic for ARP table
def parse_arp_entry(entry):
    parts = entry.split("/")
    if len(parts) == 6:
        return {
            "IP": parts[0],
            "HW_TYPE": parts[1],
            "FLAGS": parts[2],
            "HW_ADDR": parts[3],
            "MASK": parts[4],
            "DEVICE": parts[5]
        }
    return None


# Define parse logic for NAT table
def parse_nat_entry(entry):
    protocol, timeout, src_ip, src_port, dst_ip, dst_port = [None] * 6
    segments = entry.split()

    # First two elements are protocol and timeout
    if len(segments) >= 3:
        protocol = segments[0]
        timeout = segments[2]

    # Parse the first src and dst details
    for segment in segments:
        if "src=" in segment and not src_ip:
            src_ip = segment.split("src=")[-1]
        elif "dst=" in segment and not dst_ip:
            dst_ip = segment.split("dst=")[-1]
        elif "sport=" in segment and not src_port:
            src_port = segment.split("sport=")[-1]
        elif "dport=" in segment and not dst_port:
            dst_port = segment.split("dport=")[-1]

    if all([protocol, timeout, src_ip, src_port, dst_ip, dst_port]):
        return {
            "Protocol": protocol,
            "Timeout": timeout,
            "Source IP": src_ip,
            "Source Port": src_port,
            "Destination IP": dst_ip,
            "Destination Port": dst_port,
        }
    return None

# Define parse logic for dev_list_table
def parse_dev_list_entry(entry):
    parts = entry.split("/")
    if len(parts) >= 7:
        return {
            "Hostname": parts[0],
            "MAC Address": html.unescape(parts[2]),
            "IP Address": parts[1],
            "IP Allocation": parts[4],
            "Interface": parts[6],
            "Ethernet/2G/5G": parts[8],
            "Port": parts[3],
            "Connection Speed": parts[5],
            "Lease Time Remaining": parts[7]
        }
    return None

# Define parse logic for ipv6_dev_list_table
def parse_ipv6_dev_list_entry(entry):
    parts = entry.split("/")
    if len(parts) == 4:
        return {
            "Interface": parts[0],
            "MAC Address": html.unescape(parts[1]),
            "IPv6 GUAddress": html.unescape(parts[2]),
            "IPv6 LLAddress": html.unescape(parts[3]),
        }
    return None

In [ ]:
arp_url = f"http://{router_ip}/modemstatus_arptable.html"
arp_df = extract_table_data(session, arp_url, "var init_value=\"", "\";", parse_arp_entry)
print("ARP Table DataFrame:")
arp_df

In [ ]:
nat_url = f"http://{router_ip}/modemstatus_nattable.html"
nat_df = extract_table_data(session, nat_url, "var init_value=\"", "\";", parse_nat_entry)
print("NAT Table DataFrame:")
nat_df

In [ ]:
lan_status_url = f"http://{router_ip}/modemstatus_lanstatus.html"
dev_list_df = extract_table_data(session, lan_status_url, "var brinfo = '", "';", parse_dev_list_entry)
print("\nDevice List Table DataFrame:")
dev_list_df

In [ ]:
ipv6_dev_list_df = extract_table_data(session, lan_status_url, "var ipv6hostinfo = '", "';", parse_ipv6_dev_list_entry)
print("\nIPv6 Device List Table DataFrame:")
ipv6_dev_list_df

In [ ]:
import pandas as pd
import os

def add_mac_vendor_to_arp(arp_df):
    """
    Enhance the ARP DataFrame by adding a 'MAC Vendor' column based on MAC prefixes.

    Args:
        arp_df (pd.DataFrame): The ARP table DataFrame.
        project_root (str): Path to the project root where 'data/mac-vendors-export.csv' is located.

    Returns:
        pd.DataFrame: The ARP table DataFrame with the 'MAC Vendor' column added.
    """
    # Path to the MAC vendor database CSV file
    csv_file_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data", "mac-vendors-export.csv")
    
    # Load the MAC vendor database
    try:
        mac_vendor_df = pd.read_csv(csv_file_path)
    except FileNotFoundError:
        raise FileNotFoundError(f"MAC vendor database not found at {csv_file_path}. Ensure the file exists.")

    # Ensure the MAC Prefix and Vendor Name columns exist
    required_columns = {"Mac Prefix", "Vendor Name"}
    if not required_columns.issubset(mac_vendor_df.columns):
        raise ValueError(f"The CSV file must contain columns: {required_columns}")

    # Extract the first 8 characters of the MAC addresses for matching
    arp_df["MAC Prefix"] = arp_df["HW_ADDR"].str[:8].str.upper()

    # Merge the ARP table with the MAC vendor database
    arp_df = arp_df.merge(mac_vendor_df, how="left", left_on="MAC Prefix", right_on="Mac Prefix")

    # Rename 'Vendor Name' to 'MAC Vendor' for clarity and drop 'MAC Prefix'
    arp_df.rename(columns={"Vendor Name": "MAC Vendor"}, inplace=True)
    arp_df.drop(columns=["Mac Prefix"], inplace=True)

    return arp_df


In [ ]:
add_mac_vendor_to_arp(arp_df)